# Sorteio de amigo secreto

Este programa recebe um dicionário com a chave sendo o conjunto de participantes, e no conteúdo o e-mail do participante, o grupo familiar, e uma indicadora se é criança ou não (que eu não usei ainda).

- O programa sorteia as pessoas
- Você pode configurar se permite ou não sorteios dentro do grupo familiar
- O programa envia um e-mail para cada participante com o resultado através da sua conta do GMail.

## Teste do envio de e-mail

O primeiro passo é abrir uma sessão na sua conta G-Mail. Pra isso, você vai precisar de uma senha de app do gmail.

Para criar a sua senha de app faça o seguinte:

1. Acesse sua Conta do Google > Segurança
2. Em "Como fazer login no Google", selecione Senhas de app
3. Na parte inferior, escolha Selecionar app e escolha o app que você está usando e escolha Selecionar dispositivo e escolha o dispositivo que você está usando e Gerar
4. Siga as instruções para inserir a senha de app. Ela é o código de 16 caracteres na barra amarela do seu dispositivo.
5. Toque em Concluído.

Em caso de problemas acesse [aqui](https://support.google.com/accounts/answer/185833?p=InvalidSecondFactor&visit_id=637737806610866570-556942547&rd=1)

Esse script abaixo testa a conexão:

In [257]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
mail_content = '''Hello,
This is a simple mail. There is only text, no attachments are there The mail is sent using Python SMTP library.
Thank You
'''
#Senha do seu g-mail - a senha de app que vc configurou na célula acima
sender_pass = '************' # é aqui que você cola a senha de app do passo acima

#Dados de endereço a enviar, endereço do remetente, etc
message = MIMEMultipart()

message['From'] = 'sua_conta@gmail.com' # claro que você vai precisar colocar um endereço válido aqui
message['To'] = 'destinatario@gmail.com'# claro que você vai precisar colocar um endereço válido aqui
message['Subject'] = 'A test mail sent by Python. It has an attachment.'   #The subject line

#Corpo do e-mail
message.attach(MIMEText(mail_content, 'plain'))

#Criar uma sessão SMTP
try:
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password

    session.sendmail(sender_address, receiver_address, message.as_string())
    session.quit()
    print('Mensagem enviada de {0} para {1}'.format(message['From'], message['to']))
except:
    print('Deu algo errado com a conexão')



Mensagem enviada de sua_conta@gmail.com para destinatario@gmail.com


## Lista de participantes

Aqui vamos criar um dicionário onde as chaves são os nomes dos participantes. O valor correspondente é uma lista com os dados nessa ordem:

- e-mail do participante
- grupo familiar (pode ser um string, um número etc...)
- um booleano indicando se é adulto ou criança (não estou usando ainda...)

In [258]:
amigo_secreto = {'João': ['participante1@gmail.com', 1, True],
                 'Gisélia': ['participante2@gmail.com', 1, True],
                 'Ian': ['participante3@gmail.com', 1, False],
                 'Alice': ['participante4@gmail.com', 1, False],
                 'Olivia': ['participante5@gmail.com',1,False],
                 
                 'Pedro': ['participante6@gmail.com', 2, True],
                 'Themis': ['participante7@gmail.com',2,True],
                 'Felipe': ['participante8@gmail.com',2,False],
                 'Nico': ['participante9@gmail.com',2,False],
                 
                 'Mariana': ['participante10@gmail.com',3,True],
                 'Alberto': ['participante11@alopes.com',3,True],
                 'Luisa': ['participante12@gmail.com',3,False],
                 'Teresa': ['participante13@gmail.com',3,False],
                 
                 'Vovó Ana': ['participante14@ama.org.br',4,True],
                 'Vovô Samuel': ['participante15@gmail.com',4,True],
                 'Tio Gui': ['participante16@gmail.com',4,True]
                }
import copy

am_sec = copy.deepcopy(amigo_secreto)
amigo_secreto

{'João': ['participante1@gmail.com', 1, True],
 'Gisélia': ['participante2@gmail.com', 1, True],
 'Ian': ['participante3@gmail.com', 1, False],
 'Alice': ['participante4@gmail.com', 1, False],
 'Olivia': ['participante5@gmail.com', 1, False],
 'Pedro': ['participante6@gmail.com', 2, True],
 'Themis': ['participante7@gmail.com', 2, True],
 'Felipe': ['participante8@gmail.com', 2, False],
 'Nico': ['participante9@gmail.com', 2, False],
 'Mariana': ['participante10@gmail.com', 3, True],
 'Alberto': ['participante11@alopes.com', 3, True],
 'Luisa': ['participante12@gmail.com', 3, False],
 'Teresa': ['participante13@gmail.com', 3, False],
 'Vovó Ana': ['participante14@ama.org.br', 4, True],
 'Vovô Samuel': ['participante15@gmail.com', 4, True],
 'Tio Gui': ['participante16@gmail.com', 4, True]}

## Montagem dos grupos familiares

Essa etapa monta os grupos familiares como um dicionário, com a chave sendo o identificador do grupo que está no dicionáiro acima, e o valor é um *set* contendo os participantes.

Vai ser usado para garantir que a pessoa não sorteie alguém do grupo familiar (caso se deseje esta configuração).

In [116]:
# monta os grupos familiares em conjuntos
grupos = {}

for pessoa in amigo_secreto.keys():
    grupo = amigo_secreto[pessoa][1]
    if grupo in grupos.keys():
        grupos[grupo].add(pessoa)
    else: grupos[grupo] = {pessoa}

grupos[1]
    

{'Alice', 'Gisélia', 'Ian', 'João', 'Olivia'}

## Sorteio das pessoas

Agora vamos sortear as pessoas.

Tem essa opção ```autosorteio = False```. Se ela for ```False``` não será possível uma pessoa "se tirar".
```DentroDoGrupo = False``` não vai permitir sorteios dentro do grupo familiar.

O resultado do sorteio vai ficar armazenado na 4a posição da lista correspondente a cada chave do dicionário ```amigo_secreto```. Bem, se eu sofisticar mais você certamente vai conseguir rackear, mas eu sei que você não vai querer abrir esse dicionário mesmo, afinal estraga a graça né? ;)

In [242]:
import random

autosorteio=False
DentroDoGrupo=False

urna = set(amigo_secreto.keys())

resortear = True
cont=0

while resortear == True:
    amigo_secreto = copy.deepcopy(am_sec)
    urna = set(amigo_secreto.keys())
    resortear=False
    
    for pessoa in amigo_secreto.keys():
        # Monta a urna válida    
        urna_valida = urna.copy()
        urna_valida.discard(pessoa)

        # Remove da urna válida pessoas do grupo caso opção ativa
        if DentroDoGrupo==False:
            num_grupo = amigo_secreto[pessoa][1]
            grupo = grupos[num_grupo]
            urna_valida = urna_valida - grupo

        #Sorteia o 'papelzinho'
        if len(urna_valida)>0:
            papelzinho = random.sample(urna_valida,1)[0]
            urna.discard(papelzinho)
        else: 
            resortear = True
            cont+=1
        amigo_secreto[pessoa].append(papelzinho)
        papelzinho='branco'

print(resortear)
print(f'cont - {cont}')

# print(f'{pessoa} - {papelzinho}')


False
cont - 0


## Comunicar o sorteio

Agora vamos estabelecer uma comunicação com o seu gmail, em seguida vamos enviar todos os e-mails, e desligar a conexão.

Essa função abaixo estará dentro do loop, com a comunicação aberta fora dela. Ela recebe:

- fulano - email de "quem tirou o papelzinho da urna"
- email_fulano - o email do fulano que tirou o papelzinho da urna
- siclano - o nome do siclano que o fulano tirou

Tem uma mensagem em "código duro", que é enviada a cada pessoa indicando quem ela tirou.

In [250]:
def comunica_sorteio(fulano, email_fulano, siclano):
    mail_content = '''Olá, {0}!
    Essa é uma mensagem do seu amigo secreto! Você tirou...

    O nome da pessoa que você tirou é...
    
    é...:

    *** {1} ***
    
    Boas festas!!!
    '''
    #E-mail e senha da conta que vai enviar
    sender_address = 'juka42@gmail.com'
    receiver_address = amigo_secreto[fulano][0]
    sender_pass = 'oolvlvqoqnbndrmd'

    #Configurar o MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'Amigo Secreto (by Juka)'   #Linha de assunto
    
    # O corpo do texto
    message.attach(MIMEText(mail_content.format(fulano, siclano), 'plain'))
    
    text = message.as_string() #mensagem com todos os elementos em formato texto

    session.sendmail(sender_address, receiver_address, text) #envia o e-mail
    # fim da sessão
#     session.quit() # encerra a sessão
    print('E-mail enviado de {0} para {1} - {2}'.format(sender_address, sender_address, fulano))
    


Aqui estabelecemos a conexão e fazemos o loop dentre todos os participantes, indicando quem ele tirou.

In [251]:
session = smtplib.SMTP('smtp.gmail.com', 587) #Usar porta do gmail
session.starttls() #Habilitar segurança
session.login(sender_address, sender_pass) #Login com e-mail e senha

for pessoa in amigo_secreto.keys():
    comunica_sorteio(pessoa, amigo_secreto[pessoa][0], amigo_secreto[pessoa][3])
    
session.quit()

E-mail enviado de juka42@gmail.com para juka42@gmail.com - João
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Gisélia
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Ian
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Alice
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Olivia
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Pedro
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Themis
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Felipe
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Nico
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Mariana
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Alberto
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Luisa
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Teresa
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Vovó Ana
E-mail enviado de juka42@gmail.com para juka42@gmail.com - Vovô Samuel
E-mail env

(221, b'2.0.0 closing connection r2sm8247340vsk.28 - gsmtp')